In [2]:
import pandas as pd
import json

In [3]:

battles = pd.read_json('63471324.json')
battles = dict(battles.loc["battle_list"])['result_body']

In [5]:
len(battles)

100

In [6]:
battle_data = []

for x in range(len(battles)):
    battle_data.append(battles[x]['teamBettleInfo'])
    battle_data.append(battles[x]['teamBettleInfoenemy'])
    battle_data.append(battles[x]['my_deck'])
    battle_data.append(battles[x]['enemy_deck'])
    # battle_data.append(battles[x]['iswin'])


In [7]:
battle_data[5]

'"my_team":[{"pick_order" :1,"hero_code":"c6037","attack_damage":271,"receive_damage":50356.93,"recovery":21525,"mvp_point":500266,"artifact":"efh12","equip":["set_immune","set_speed"],"respawn":0,"mvp":1,"kill_count":0,"position":3,"grade":6,"awaken_grade":6,"attribute_cd":"light","level":60,"job_cd":"manauser"},{"pick_order" :2,"hero_code":"c1159","attack_damage":1119,"receive_damage":93121.8,"recovery":0,"mvp_point":129916,"artifact":"efw35","equip":["set_max_hp","set_speed"],"respawn":0,"mvp":0,"kill_count":0,"position":2,"grade":6,"awaken_grade":6,"attribute_cd":"wind","level":60,"job_cd":"warrior"},{"pick_order" :3,"hero_code":"c2070","attack_damage":14995,"receive_damage":59378.92,"recovery":27939,"mvp_point":280694,"artifact":"efk04","equip":["set_speed","set_max_hp"],"respawn":0,"mvp":0,"kill_count":0,"position":1,"grade":6,"awaken_grade":6,"attribute_cd":"light","level":60,"job_cd":"knight"},{"pick_order" :4,"hero_code":"c2071","attack_damage":57510,"receive_damage":107691.89

In [8]:
len(battles)

100

In [9]:
def drafts(team_1, team_2, deck_1, deck_2):
    
    data = json.loads(team_1)
    my_team_data = data["my_team"]
    df_team = pd.DataFrame(my_team_data)
    df_team = df_team.drop(['artifact', 'respawn', 'mvp', 'kill_count', 'position', 'grade', 'awaken_grade', 'attribute_cd', 'level', 'job_cd'], axis=1)
    
    data = json.loads(team_2)
    my_team_data_enemy = data["my_team"]
    df_enemy_team = pd.DataFrame(my_team_data_enemy)
    df_enemy_team = df_enemy_team.drop(['artifact', 'respawn', 'mvp', 'kill_count', 'position', 'grade', 'awaken_grade', 'attribute_cd', 'level', 'job_cd'], axis=1)
        
    my_hero_list = json.dumps(deck_1)
    data_dict = json.loads(my_hero_list)
    df_hero_list = pd.DataFrame(data_dict['hero_list'])
    
    enemy_hero_list = json.dumps(deck_2)
    data_dict = json.loads(enemy_hero_list)
    df_enemy_hero_list = pd.DataFrame(data_dict['hero_list'])
    
    # takes both side drafts into one data entry 
    result = pd.merge(df_team, df_hero_list)
    result2 = pd.merge(df_enemy_team, df_enemy_hero_list)
    draft = pd.concat([result, result2], ignore_index=True,axis=0)
    
    return draft 

In [10]:
# stores all generated drafts
drafts_list = []

# generate draft data 
i = 0
while i < len(battles)*4:
    string_data_team = '{' + battle_data[i] + '}'
    string_data_enemy = '{' + battle_data[i+1] + '}'
    string_data_my_deck = battle_data[i+2] 
    string_data_enemy_deck = battle_data[i+3]
    res = drafts(string_data_team, string_data_enemy , string_data_my_deck, string_data_enemy_deck)
    drafts_list.append(res)
    i += 4
                                                                  

In [11]:
drafts_list[30]

,pick_order,hero_code,attack_damage,receive_damage,recovery,mvp_point,equip,first_pick,mvp,ban
0,1,c2089,881,4347.33,0,57411,[set_speed],1,0,0
1,2,c2109,39472,0.00,0,77159,"[set_counter, set_max_hp]",0,1,0
2,3,c4004,0,0.00,11178,34370,"[set_def, set_max_hp, set_max_hp]",0,0,0
3,4,c5089,0,0.00,0,0,[],0,0,1
4,5,c1102,736,0.00,1366,25295,"[set_immune, set_shield]",0,0,0
5,1,c1126,0,26148.79,0,26148,[set_speed],0,0,0
6,2,c1159,0,80834.08,0,80834,[set_speed],0,0,0
7,3,c2070,0,5618.43,36184,105217,"[set_speed, set_max_hp]",0,1,0
8,4,c2071,1139,8614.00,0,9752,"[set_cri_dmg, set_def]",0,0,0
9,5,c1151,0,0.00,0,0,[],0,0,1


In [12]:
# 100 drafts to csv file
pd.concat(drafts_list).to_csv('drafts.csv')